In [1]:
import numpy as np
import tensorflow as tf

In [2]:
#Constant
batch_size = 2;
embedding_dimension = 64;
num_classes = 2;
hidden_layer_size = 32;
times_steps = 6;
element_size = 1;

In [4]:
# map
digit_to_word_map = {1:"One",2:"Two",3:"Three",4:"Four",5:"Five",
                    6:"Six",7:"Seven",8:"Eight",9:"Nine",}
digit_to_word_map[0]="PAD"

In [8]:
# Train Data Create
Sample_cnt = 10000
odd_sentences=[]
even_sentences=[]
seqlens=[]
for i in range(Sample_cnt):
    rand_seq_len = np.random.choice(range(3,7))
    seqlens.append(rand_seq_len)
    rand_odd_ints = np.random.choice(range(1,10,2), rand_seq_len)
    rand_even_ints = np.random.choice(range(2,10,2), rand_seq_len)
    #print('rand_seq_len:{} rand_odd_ints:{} rand_even_ints:{}'.format(rand_seq_len, rand_odd_ints, rand_even_ints))

    # 패딩
    if rand_seq_len < 6:
        #print(rand_odd_ints, np.append(rand_odd_ints, [0]*(6-rand_seq_len)))
        rand_odd_ints = np.append(rand_odd_ints, [0]*(6-rand_seq_len))
        rand_even_ints = np.append(rand_even_ints, [0]*(6-rand_seq_len))
    #print('  >> rand_seq_len:{} rand_odd_ints:{} rand_even_ints:{}'.format(rand_seq_len, rand_odd_ints, rand_even_ints))
    odd_sentences.append(" ".join([digit_to_word_map[r] for r in rand_odd_ints]))
    even_sentences.append(" ".join([digit_to_word_map[r] for r in rand_even_ints]))
    # print(" ".join([digit_to_word_map[r] for r in rand_odd_ints]))
    # print(" ".join([digit_to_word_map[r] for r in rand_even_ints]))
    # print('*'*100)
data = odd_sentences + even_sentences
len(odd_sentences),len(even_sentences),len(data)

# 홀수, 짝수 시퀀스의 seq 길이(2배 증폭)
seqlens*=2
odd_sentences[:5], even_sentences[:5], seqlens[:5]

(['Three Nine Seven Nine PAD PAD',
  'Five Nine One Nine PAD PAD',
  'Nine Nine Five Five Three PAD',
  'Seven Five Seven PAD PAD PAD',
  'Five Three Five PAD PAD PAD'],
 ['Two Eight Six Six PAD PAD',
  'Eight Two Two Eight PAD PAD',
  'Two Eight Eight Eight Six PAD',
  'Two Four Two PAD PAD PAD',
  'Six Eight Six PAD PAD PAD'],
 [4, 4, 5, 3, 3])

In [10]:
# 단어를 인덱스에 매핑
word2index_map = {}
index = 0
for sent in data:
    #print(sent)
    for word in sent.lower().split():
        #print(word)
        if word not in word2index_map:
            word2index_map[word] = index
            index += 1
        #print(word2index_map)

# 역방향 매핑
index2word_map = {index: word for word, index in word2index_map.items()}
vocabulary_size = len(index2word_map)

print('word2index_map :{}'.format(word2index_map))
print('index2word_map :{}'.format(index2word_map))
print('vocabulary_size :{}'.format(vocabulary_size))


word2index_map :{'three': 0, 'nine': 1, 'seven': 2, 'pad': 3, 'five': 4, 'one': 5, 'two': 6, 'eight': 7, 'six': 8, 'four': 9}
index2word_map :{0: 'three', 1: 'nine', 2: 'seven', 3: 'pad', 4: 'five', 5: 'one', 6: 'two', 7: 'eight', 8: 'six', 9: 'four'}
vocabulary_size :10


In [11]:
# one hot encoding : 짝수=[1,0], 홀수=[0,1]
labels = [1]*Sample_cnt + [0]*Sample_cnt
for i in range(len(labels)):
    label = labels[i]
    one_hot_encoding = [0]*2
    one_hot_encoding[label] = 1
    labels[i] = one_hot_encoding
    #print(i, label, one_hot_encoding)
print('앞쪽')
print('   >>> date : {}'.format(data[:3]))
print('   >>> one-hot-encoding : {}'.format(labels[:3]))
print('뒤쪽')
print('   >>> date : {}'.format(data[Sample_cnt*2-4:-1]))
print('   >>> one-hot-encoding : {}'.format(labels[Sample_cnt*2-4:-1]))

앞쪽
   >>> date : ['Three Nine Seven Nine PAD PAD', 'Five Nine One Nine PAD PAD', 'Nine Nine Five Five Three PAD']
   >>> one-hot-encoding : [[0, 1], [0, 1], [0, 1]]
뒤쪽
   >>> date : ['Six Eight Six PAD PAD PAD', 'Four Eight Eight Four Two PAD', 'Six Six Two Two PAD PAD']
   >>> one-hot-encoding : [[1, 0], [1, 0], [1, 0]]


In [14]:
# Train Set, Test Set 만들기
data_indices = list(range(len(data)))
np.random.shuffle(data_indices)
data_shuffle = np.array(data)[data_indices]
labels_shuffle = np.array(labels)[data_indices]
seqlens_shuffle = np.array(seqlens)[data_indices]
#print('data_shuffle : {}'.format(data_shuffle[:5]))
#print('labels_shuffle : {}'.format(labels_shuffle[:5]))
#print('seqlens_shuffle : {}'.format(seqlens_shuffle[:5]))

train_x = data_shuffle[:Sample_cnt]
train_y = labels_shuffle[:Sample_cnt]
train_seqlens = seqlens_shuffle[:Sample_cnt]

test_x = data_shuffle[Sample_cnt:]
test_y = labels_shuffle[Sample_cnt:]
test_seqlens = seqlens_shuffle[Sample_cnt:]

print('Train Length : {}(x) {}(y) {}(seqlen)'.format(len(train_x),len(train_y),len(train_seqlens),))
print('Test  Length : {}(x) {}(y) {}(seqlen)'.format(len(test_x),len(test_y),len(test_seqlens),))

Train Length : 10000(x) 10000(y) 10000(seqlen)
Test  Length : 10000(x) 10000(y) 10000(seqlen)


In [15]:
# bath_size만큼 데이터를 Random하게 제공
def get_sentence_batch(batch_size, data_x,
                       data_y, data_seqlens):
    instance_indices = list(range(len(data_x)))
    np.random.shuffle(instance_indices)
    batch = instance_indices[:batch_size]
    x = [[word2index_map[word] for word in data_x[i].lower().split()]
         for i in batch]
    y = [data_y[i] for i in batch]
    seqlens = [data_seqlens[i] for i in batch]
    return x, y, seqlens

In [17]:
# train data의 placeholder
_inputs = tf.placeholder(tf.int32, shape=[batch_size, times_steps])
_labels = tf.placeholder(tf.int32, shape=[batch_size, num_classes])
_seqlens = tf.placeholder(tf.int32, shape=[batch_size])

In [18]:
# embedding
# 고차원의 vocabulary Sequence를 저차원의 vector로 변형한다.
with tf.name_scope("embeddings"):
    # embeddings : <tf.Variable 'embeddings/embedding:0' shape=(10, 64) dtype=float32_ref>
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_dimension], -1.0, 1.0), name='embedding')
    # embed : <tf.Tensor 'embeddings/embedding_lookup:0' shape=(2, 6, 64) dtype=float32>
    embed = tf.nn.embedding_lookup(embeddings, _inputs)

In [19]:
with tf.name_scope("lstm"):
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(hidden_layer_size, forget_bias=1.0)
    with tf.variable_scope("lstm_kernel", reuse=tf.AUTO_REUSE):
        outputs, states = tf.nn.dynamic_rnn(lstm_cell, embed, sequence_length = _seqlens, dtype=tf.float32)

In [21]:
# 최종상태를 뽑아서 선형계측에 적용
weights = {'linear_layer' : tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes], mean=0, stddev=.01))}
biases = {'linear_layer' : tf.Variable(tf.truncated_normal([num_classes], mean=0, stddev=.01))}
final_output = tf.matmul(states[1], weights["linear_layer"]) + biases["linear_layer"]

In [23]:
# Loss Function / Train
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=final_output, labels=_labels))
train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)

In [24]:
# Accuracy Define
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(_labels, 1), tf.argmax(final_output, 1))
    accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))*100

In [25]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        batch_x, batch_y, batch_seqlen = get_sentence_batch(batch_size, train_x, train_y, train_seqlens)
        sess.run(train_step, feed_dict={_inputs: batch_x, _labels: batch_y, _seqlens: batch_seqlen})

        if i == 0 or (i+1) % 100 == 0:
            acc, loss, = sess.run([accuracy, cross_entropy], feed_dict={_inputs: batch_x, _labels: batch_y, _seqlens: batch_seqlen})
            print("Iter " + str(i+1) + ", Minibatch Loss= " +
                  "{:.6f}".format(loss) + ", Training Accuracy= " +
                  "{:.5f}".format(acc))
    for test_batch in range(5):
        x_test, y_test, seqlen_test = get_sentence_batch(batch_size,
                                                         test_x, test_y,
                                                         test_seqlens)
        batch_pred, batch_acc = sess.run([tf.argmax(final_output, 1), accuracy],
                                         feed_dict={_inputs: x_test,
                                                    _labels: y_test,
                                                    _seqlens: seqlen_test})
        print("Test batch accuracy %d: %.5f" % (test_batch, batch_acc))
    output_example = sess.run([outputs], feed_dict={_inputs: x_test,
                                                    _labels: y_test,
                                                    _seqlens: seqlen_test})
    states_example = sess.run([states[1]], feed_dict={_inputs: x_test,
                                                      _labels: y_test,
                                                      _seqlens: seqlen_test})
print("Complete")

Iter 1, Minibatch Loss= 0.678877, Training Accuracy= 100.00000
Iter 100, Minibatch Loss= 0.567000, Training Accuracy= 100.00000
Iter 200, Minibatch Loss= 0.021101, Training Accuracy= 100.00000
Iter 300, Minibatch Loss= 0.000472, Training Accuracy= 100.00000
Iter 400, Minibatch Loss= 0.000031, Training Accuracy= 100.00000
Iter 500, Minibatch Loss= 0.000003, Training Accuracy= 100.00000
Iter 600, Minibatch Loss= 0.000001, Training Accuracy= 100.00000
Iter 700, Minibatch Loss= 0.000001, Training Accuracy= 100.00000
Iter 800, Minibatch Loss= 0.000001, Training Accuracy= 100.00000
Iter 900, Minibatch Loss= 0.000000, Training Accuracy= 100.00000
Iter 1000, Minibatch Loss= 0.000000, Training Accuracy= 100.00000
Iter 1100, Minibatch Loss= 0.000000, Training Accuracy= 100.00000
Iter 1200, Minibatch Loss= 0.000000, Training Accuracy= 100.00000
Iter 1300, Minibatch Loss= 0.000000, Training Accuracy= 100.00000
Iter 1400, Minibatch Loss= 0.000000, Training Accuracy= 100.00000
Iter 1500, Minibatch L